# About this example

This example shows how you can run the DistilBERT model. You can learn more about it on this [Hugging Face page](https://huggingface.co/docs/transformers/model_doc/distilbert).

The model will be left untrained for demonstration purposes.

## Install dependencies

Install the dependencies this example needs.

In [1]:
!pip install -q transformers[onnx]

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


Install the latest version of BlindAI.

In [3]:
!pip install --quiet blindai

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


## Prepare the model

In [5]:
from transformers import DistilBertForSequenceClassification

# Load the model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

Then, export the model

In [6]:
from transformers import DistilBertTokenizer
import torch

# Create dummy input for export
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
sentence = "I love AI and privacy!"
inputs = tokenizer(sentence, padding = "max_length", max_length = 8, return_tensors="pt")["input_ids"]

# Export the model
torch.onnx.export(
	model, inputs, "./distilbert-base-uncased.onnx",
	export_params=True, opset_version=11,
	input_names = ['input'], output_names = ['output'],
	dynamic_axes={'input' : {0 : 'batch_size'},
	'output' : {0 : 'batch_size'}})

This may take a little while.

# Upload model to inference server

Please make sure the server is running. To launch the server, refer to the [Launching the server](https://docs.mithrilsecurity.io/getting-started/quick-start/run-the-blindai-server) documentation page.

If you wish to run this example in hardware mode, you need to prepare the `host_server.pem` and `policy.toml` files. Learn more on the [Deploy on Hardware](https://docs.mithrilsecurity.io/getting-started/deploy-on-hardware) documentation page. 

In [7]:
from blindai.client import BlindAiClient, ModelDatumType

# Launch client
client = BlindAiClient()

# Simulation mode
client.connect_server(addr="localhost", simulation=True)
# Hardware mode
# client.connect_server(addr="localhost", policy="./policy.toml", certificate="./host_server.pem")

Then, upload the model

In [9]:
client.upload_model(model="./distilbert-base-uncased.onnx", shape=inputs.shape, dtype=ModelDatumType.I64)

# Send data for prediction

In [11]:
from transformers import DistilBertTokenizer

# Prepare the inputs
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
sentence = "I love AI and privacy!"
inputs = tokenizer(sentence, padding = "max_length", max_length = 8)["input_ids"]

In [12]:
response = client.run_model(inputs)
response.output

[0.1299879103899002, 0.016796546056866646]

Here we can compare the results against the original prediction.

In [13]:
model(torch.tensor(inputs).unsqueeze(0)).logits.detach()

tensor([[0.1300, 0.0168]])